# NB1.5 — Streaming vs Batch (WebSockets & Polling)

En esta práctica implementamos **Streaming** con WebSockets y **Batch frecuente** con HTTP Polling. Guardaremos todo en **JSON Lines** en `../../data/` y compararemos latencia y continuidad.

**Objetivos**
- Entender diferencias clave **Batch vs Streaming**.
- Ejecutar un cliente **WebSocket** para ingesta continua.
- Ejecutar **HTTP Polling** como micro-batch.
- Comparar resultados en pandas.


## 1) Dependencias
Instalamos librerías requeridas para los scripts en `src/streaming/`.

In [1]:
!pip install -q websockets requests pandas

## 2) Streaming (WebSocket)

**Salida:** `../../data/stream_ws_YYYY-MM-DD.jsonl`

In [2]:
# Limita duración/eventos para la clase
!WS_MAX_EVENTS=150 WS_MAX_SECONDS=120 python ../../src/streaming/stream_dual_ws.py

python3: can't open file '/content/../../src/streaming/stream_dual_ws.py': [Errno 2] No such file or directory


## 3) Polling (HTTP)

**Salida:** `../../data/poll_bitcoin_YYYY-MM-DD.jsonl`

In [3]:
!python ../../src/streaming/poll_coincap_http.py

python3: can't open file '/content/../../src/streaming/poll_coincap_http.py': [Errno 2] No such file or directory


## 4) Cargar y comparar en pandas
Leemos ambos `.jsonl` y observamos diferencias de granularidad/latencia (aprox.).

In [9]:
from pathlib import Path
import pandas as pd

data_dir = Path("../../data")

# Buscar archivos
stream_files = sorted(data_dir.glob("stream_ws_*.jsonl"))
if not stream_files:
    print("No se encontraron archivos de stream")
else:
    stream_file = stream_files[-1]
    df_stream = pd.read_json(stream_file, lines=True)
    print("STREAM:", stream_file)
    display(df_stream.head())

# POLL
poll_files = sorted(list(data_dir.glob("poll_binance_*.jsonl")) + list(data_dir.glob("poll_coincap_*.jsonl")))
if not poll_files:
    print("No se encontraron archivos de poll")
else:
    poll_file = poll_files[-1]
    df_poll = pd.read_json(poll_file, lines=True)
    print("POLL:", poll_file)
    display(df_poll.head())



No se encontraron archivos de stream
No se encontraron archivos de poll


## 5) Reflexión (respuestas cortas)
**¿Quién tiene menor latencia?**  Stream: entrega los datos en tiempo real o casi inmediato, por lo que la latencia es mucho menor que polling.

**¿Qué pasa si el stream se cae?**
Se pierden los datos que se generan mientras el stream está caído, a menos que haya un mecanismo de reintento o buffering.

**¿Cuál genera más duplicados o huecos temporales?**
Polling: puede generar duplicados si se consulta demasiado seguido o huecos si se consulta con retraso, porque no recibe los datos de manera continua como el stream.